Jigsaw toxic comment classification challenge kaggle

In [80]:
import os 
import sys
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import roc_auc_score

In [81]:
max_seq_len = 100
max_vocab_size = 20000
embedding_dim = 100
val_split = 0.2
batch_size = 128
epochs = 10

In [83]:
#loading wordvecs from glove
word2vec = {}
with open(os.path.join('F:/my files/python files/neural networks/keras projects/data/glove/glove.6B.%sd.txt' %embedding_dim), encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
print('founf %s word vectors.' %len(word2vec))       

founf 400000 word vectors.


In [84]:
#loading the data
data = pd.read_csv('F:/my files/python files/neural networks/keras projects/data/toxic comments/train.csv')
comment = data['comment_text']
comment = comment.fillna('NIL')  # Fills up empty spaces
comment = comment.values  # converts to matrix
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
targets = data[labels]
targets = targets.values
print('max length:', max(len(s) for s in comment))
print('min length:', min(len(s) for s in comment))

max length: 5000
min length: 6


In [32]:
#creating tokens and converting strings to integers
token = Tokenizer(num_words=max_vocab_size)
token.fit_on_texts(comment)
seq = token.texts_to_sequences(comment)
print(seq[3])

[57, 7, 228, 97, 54, 328, 1436, 15, 2133, 7, 6024, 22, 1, 123, 2502, 56, 16, 513, 15, 25, 5, 4236, 3, 1327, 3, 9762, 7, 67, 1, 277, 85, 122, 13503, 37, 9, 51, 19, 42, 10, 1, 1460, 138, 1257, 2153, 426, 1257, 319, 7, 40, 33, 9, 513, 15, 22, 46, 48, 310, 103, 113, 22, 6, 18, 54, 4442, 12, 2394, 275, 15, 277, 25, 105, 2, 33, 11, 206, 45, 267, 35, 69, 41, 542, 2, 16, 5, 5956, 15, 80, 12, 385, 37, 7, 645, 41, 85, 16, 5, 2024, 386, 5, 3057, 3245, 76, 78, 424, 10, 1, 502, 640, 3597, 28, 98, 23, 3542, 4901]


In [85]:
word2idx = token.word_index
print('unique tokens:', len(word2idx))
print(type(word2idx))

unique tokens: 210337
<class 'dict'>


In [86]:
padded_data = pad_sequences(seq, maxlen=max_seq_len, truncating='pre')
print(padded_data.shape)
print(padded_data[3])

(159571, 100)
[ 2502    56    16   513    15    25     5  4236     3  1327     3  9762
     7    67     1   277    85   122 13503    37     9    51    19    42
    10     1  1460   138  1257  2153   426  1257   319     7    40    33
     9   513    15    22    46    48   310   103   113    22     6    18
    54  4442    12  2394   275    15   277    25   105     2    33    11
   206    45   267    35    69    41   542     2    16     5  5956    15
    80    12   385    37     7   645    41    85    16     5  2024   386
     5  3057  3245    76    78   424    10     1   502   640  3597    28
    98    23  3542  4901]


In [87]:
num_words = min(max_vocab_size, len(word2idx)+1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word2idx.items():
    if i < max_vocab_size:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
           
embedding_layer = Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_seq_len, trainable=False)            
            

In [88]:
# neural network model
inp = Input(shape=(max_seq_len,))
x = embedding_layer(inp)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
oup = Dense(6, activation='sigmoid')(x)

In [92]:
model = Model(inputs=inp, outputs=oup)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 100)               0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 100, 100)          2000000   
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 98, 128)           38528     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 32, 128)           0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 30, 128)           49280     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 10, 128)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 8, 128)            49280     
__________

In [97]:
checkpoint = ModelCheckpoint(filepath='comm.weights.best.hdf5', verbose=1, save_best_only=True)
model.fit(padded_data, targets, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint], validation_split=val_split, verbose=1)

Train on 127656 samples, validate on 31915 samples
Epoch 1/10


   128/127656 [..............................] - ETA: 49:03 - loss: 0.7489 - acc: 0.3294

   256/127656 [..............................] - ETA: 26:24 - loss: 0.6067 - acc: 0.6341

   384/127656 [..............................] - ETA: 18:21 - loss: 0.4653 - acc: 0.7470

   512/127656 [..............................] - ETA: 14:20 - loss: 0.4024 - acc: 0.8031

...

127488/127656 [============================>.] - ETA: 0s - loss: 0.0505 - acc: 0.9821



127616/127656 [============================>.] - ETA: 0s - loss: 0.0505 - acc: 0.9820

127656/127656 [==============================] - 201s 2ms/step - loss: 0.0505 - acc: 0.9821 - val_loss: 0.0938 - val_acc: 0.9773



Epoch 00010: val_loss did not improve


In [100]:
y_hat = model.predict(padded_data)
auc = roc_auc_score(targets, y_hat) * 100
print(auc)

97.2060409678
